In [11]:
import pandas as pd
import numpy as np
import sqlalchemy as sqla
from io import StringIO
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='zhenhangliu', api_key='vNe3aFwBSfb1pV4LNLwO')
mapbox_access_token = 'pk.eyJ1IjoiemhlbmhhbmdsaXUiLCJhIjoiY2pvemZremhyMG1sOTN2cDRiMjJ4bmR0MSJ9.UHRGP-HIJHJJA7np0AxlbQ'
conn = sqla.create_engine('sqlite:///yelp.sqlite')


In [6]:
business=pd.read_sql_table("business",conn)
business

,address,categories,city,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
1,,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV
2,1335 rue Beaubien E,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC
3,211 W Monroe St,"Insurance, Financial Services",Phoenix,1,33.449999,-112.076979,Geico Insurance,,85003,8,1.5,AZ
4,2005 Alyth Place SE,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,1,51.035591,-114.027366,Action Engine,,T2H 0N5,4,2.0,AB
5,"20235 N Cave Creek Rd, Ste 1115","Coffee & Tea, Food",Phoenix,1,33.671375,-112.030017,The Coffee Bean & Tea Leaf,,85024,63,4.0,AZ
6,631 Bloor St W,"Food, Bakeries",Toronto,0,43.664378,-79.414424,Bnc Cake House,Koreatown,M6G 1K8,7,4.0,ON
7,"3417 Derry Road E, Unit 103","Restaurants, Thai",Mississauga,1,43.712946,-79.632763,Thai One On,Ridgewood,L4T 1A8,7,2.0,ON
8,1440 N. Dysart Ave,"Mexican, Restaurants",Avondale,1,33.448106,-112.341302,Filiberto's Mexican Food,,85323,40,2.5,AZ
9,209 Oakland Ave,"Flowers & Gifts, Gift Shops, Shopping",Pittsburgh,1,40.441421,-79.956457,Maggie & Stella's Gifts,Oakland,15213,3,3.5,PA


In [12]:
business.replace("",np.nan,inplace=True)

In [13]:
location=business[business["longitude"].notnull() & business["latitude"].notnull()]


In [ ]:
# data = [
#     go.Scattermapbox(
#         lat=location["latitude"],
#         lon=location["longitude"],
#         mode='markers',
#         marker=dict(
#             opacity = 0.6,
#             size=10,
#         ),
#         text=location["name"]+"<br>Stars: "+location["stars"].astype(str)\
#         +"<br>Review: "+location["review_count"].astype(str)+"<br>"+location["city"].fillna('')+","\
#         +location["state"].fillna('')+" "+location["postal_code"].fillna(''),
#     ) 
# ]

# layout = go.Layout(
#     autosize=True,
#     title="Yelp Data Map Visualization",
#     hovermode='closest',
#     mapbox=dict(
#         accesstoken=mapbox_access_token,
#         bearing=0,
#         center=dict(
#             lat=location["latitude"].median(),
#             lon=location["longitude"].median()
#         ),
#         pitch=0,
#         zoom=.5,
#         style= 'mapbox://styles/zhenhangliu/cjp0dy6xg05rt2rod3gviy342'
#     ),
# )
# fig=dict(data=data,layout=layout)
# py.iplot(fig,filename="Yelp Data")

In [66]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
clf = LocalOutlierFactor(n_neighbors=1000,metric="euclidean",contamination=0.0003)


In [67]:
AZ_outlier=pd.DataFrame(clf.fit_predict(location[location["state"]=="AZ"][["latitude","longitude"]]))

In [68]:
AZ=location[location["state"]=="AZ"].copy()
AZ.reset_index(inplace=True)
AZ["outlier"]=AZ_outlier
AZ.set_index("business_id",inplace=True)
AZ.index.name = None


In [69]:
data = [
    go.Scattermapbox(
        lat=location["latitude"],
        lon=location["longitude"],
        mode='markers',
        marker=dict(
            opacity = 0.6,
            size=10,
        ),
        text=location["name"]+"<br>Stars: "+location["stars"].astype(str)\
        +"<br>Review: "+location["review_count"].astype(str)+"<br>"+location["city"].fillna('')+","\
        +location["state"].fillna('')+" "+location["postal_code"].fillna(''),
        name=location["outlier"].iloc[0].astype(str),
    ) for location in (AZ[AZ["outlier"]==i] for i in AZ["outlier"].unique())
]

layout = go.Layout(
    autosize=True,
    title="Yelp Data Map Visualization",
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=location["latitude"].median(),
            lon=location["longitude"].median()
        ),
        pitch=0,
        zoom=.5,
        style= 'mapbox://styles/zhenhangliu/cjp0dy6xg05rt2rod3gviy342'
    ),
)
fig=dict(data=data,layout=layout)
py.iplot(fig,filename="Yelp Data")

/home/zackliu/anaconda3/lib/python3.6/site-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.

/home/zackliu/anaconda3/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [ ]:
[location["outlier"].iloc[0] for location in (AZ[AZ["outlier"]==i] for i in AZ["outlier"].unique())]

In [15]:
AZ=location[location["state"]=="AZ"].copy()
from sklearn.neighbors import LocalOutlierFactor

In [33]:
AZ["outlier"].unique()

array([ 1, -1])